In [1]:
import numpy as np
import pandas as pd

def binary_entropy(e):
    e = np.clip(e, 1e-12, 1 - 1e-12)
    return -e * np.log2(e) - (1 - e) * np.log2(1 - e)

np.random.seed(42)
n_samples = 40000

# Mesafe (km)
distance_km = np.random.uniform(0, 100, n_samples)

# Kanal kaybı (dB)
channel_loss_dB = distance_km * 0.2 + np.random.normal(0, 0.5, n_samples)
channel_loss_dB = np.clip(channel_loss_dB, 0, 25)

# Transmission: önerilen üssel azalma
transmission = 0.8 * np.exp(-channel_loss_dB / 10)
transmission = np.clip(transmission, 0.05, 0.8)

# Photon rate: önerilen aralıkta, kanal kaybına bağlı
photon_rate = 1e7 * np.exp(-channel_loss_dB / 10)
photon_rate = np.clip(photon_rate, 1e5, 1e7)

# Dark count rate: log-normal dağılım
dark_count_rate = np.random.lognormal(mean=-8, sigma=1, size=n_samples)
dark_count_rate = np.clip(dark_count_rate, 1e-7, 1e-3)

# Decoy mean photon: transmission + normal gürültü
decoy_mean_photon = 0.1 + 0.8 * transmission + np.random.normal(0, 0.05, n_samples)
decoy_mean_photon = np.clip(decoy_mean_photon, 0.1, 0.9)

# QBER: yeni base_error ve katsayılar
base_error = 0.015  # 0.01 - 0.02 arası
qber = base_error + 0.015 * (channel_loss_dB / channel_loss_dB.max()) + 0.25 * dark_count_rate
qber = np.clip(qber, 0.01, 0.15)

# Hata düzeltme katsayısı f (1.1 - 1.3)
f = np.random.uniform(1.1, 1.3, n_samples)

# Ham anahtar hızı R
R = photon_rate * transmission

# Binary entropy H(QBER)
H_qber = binary_entropy(qber)

# SKR (Devetak-Winter Bound sade hali)
SKR = R * (1 - f * H_qber - H_qber)
SKR = np.clip(SKR, 0, None)

data = {
    "distance_km": distance_km,
    "channel_loss_dB": channel_loss_dB,
    "transmission": transmission,
    "photon_rate": photon_rate,
    "dark_count_rate": dark_count_rate,
    "decoy_mean_photon": decoy_mean_photon,
    "qber": qber,
    "error_correction_f": f,
    "SKR": SKR,
}

df = pd.DataFrame(data)

df.to_csv("qkd_final_revised_dataset_40k.csv", index=False)


   distance_km  channel_loss_dB  transmission   photon_rate  dark_count_rate  \
0    37.454012         7.207286      0.389118  4.863978e+06         0.000178   
1    95.071431        18.873580      0.121177  1.514715e+06         0.000375   
2    73.199394        14.554837      0.186630  2.332875e+06         0.000197   
3    59.865848        11.873664      0.244019  3.050235e+06         0.000227   
4    15.601864         2.797916      0.604753  7.559413e+06         0.000092   

   decoy_mean_photon      qber  error_correction_f           SKR  
0           0.475114  0.020128            1.203728  1.299719e+06  
1           0.134573  0.028407            1.137458  1.104423e+05  
2           0.245799  0.025316            1.123673  2.778952e+05  
3           0.292673  0.023432            1.271653  4.732731e+05  
4           0.590419  0.016997            1.297364  3.266864e+06  


In [2]:
import os
print(os.getcwd())

C:\Users\yucen\DepresyonModel
